In [1]:
import feedparser
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd

url = "http://news.google.com/news?q=covid&hl=en-US&sort=date&gl=US&num=10&output=rss"

class ParseFeed():
    def __init__(self, url):
        self.feed_url = url
        
    def clean(self, html):
        '''
        Get the text from html and do some cleaning
        '''
        soup = BeautifulSoup(html)
        text = soup.get_text()
        text = text.replace('\xa0', ' ')
        return text

    def parse(self):
        newsDescription = []
        newsUrl = []
        newsDate = []
        
        '''
        Parse the URL, and print all the details of the news 
        '''        
        feeds = feedparser.parse(self.feed_url).entries
        
        for f in feeds:
            newsDescription.append(self.clean(f.get("description", "")))
            newsUrl.append(f.get("link", ""))
            newsDate.append(f.get("published", ""))
            
        df = pd.DataFrame({'Description':newsDescription, 'URL':newsUrl, 'Date Published':newsDate})            
        return df
    
htmlOutput = ""
feed = ParseFeed(url)
df = feed.parse()

In [2]:
df['Date'] = pd.to_datetime(df['Date Published'])
df = df.head(10).sort_values(by = ['Date'], ascending=False)

# Saving data to html output
htmlOutput = """<html><title>COVID-19 Daily News</title><body><style>
		body {
		  background-image: url('background.jpg');
		  background-repeat: no-repeat;
		  background-attachment: fixed;
		  background-size: cover;
		}
	  </style><center><h1>COVID-19 Daily News</h1></center><dl>"""

for index,row in df.iterrows():
    htmlOutput = htmlOutput + "<dt>" + row['Description'] + "</dt><dd>" + row['Date Published'] + "</dd><dd><a href =" + row['URL'] + ">" + row['URL'] + "</a></dd><br>"
htmlOutput = htmlOutput + "</dl></body></html>"
text_file = open("home.html", "w")
text_file.write(htmlOutput)
text_file.close()

df.to_csv('data.csv', index=False)
df

,Description,URL,Date Published,Date
4,Most Americans probably won't be able to get a...,https://www.cnn.com/2020/09/16/health/us-coron...,"Wed, 16 Sep 2020 21:18:00 GMT",2020-09-16 21:18:00+00:00
7,Turkish scientists and physicians face crimina...,https://www.sciencemag.org/news/2020/09/turkis...,"Wed, 16 Sep 2020 19:30:00 GMT",2020-09-16 19:30:00+00:00
3,An 'uncoordinated' immune response may explain...,https://www.sciencemag.org/news/2020/09/uncoor...,"Wed, 16 Sep 2020 18:00:00 GMT",2020-09-16 18:00:00+00:00
5,"From leprosy to COVID-19, how stigma makes it ...",https://www.sciencemag.org/news/2020/09/lepros...,"Wed, 16 Sep 2020 18:00:00 GMT",2020-09-16 18:00:00+00:00
1,"Eli Lilly reports first, promising results for...",https://www.sciencemag.org/news/2020/09/eli-li...,"Wed, 16 Sep 2020 15:15:00 GMT",2020-09-16 15:15:00+00:00
0,COVID-19 Daily Update 9-16-2020 West Virginia...,https://dhhr.wv.gov/News/2020/Pages/COVID-19-D...,"Wed, 16 Sep 2020 14:15:07 GMT",2020-09-16 14:15:07+00:00
6,Coronavirus (COVID-19) Update: FDA Publishes C...,https://www.fda.gov/news-events/press-announce...,"Tue, 15 Sep 2020 21:30:00 GMT",2020-09-15 21:30:00+00:00
2,Coronavirus (COVID-19) Update: Daily Roundup S...,https://www.fda.gov/news-events/press-announce...,"Tue, 15 Sep 2020 20:30:00 GMT",2020-09-15 20:30:00+00:00
8,NIH awards contracts to develop innovative dig...,https://www.nih.gov/news-events/news-releases/...,"Tue, 15 Sep 2020 15:21:36 GMT",2020-09-15 15:21:36+00:00
9,Coronavirus (COVID-19) Update: Daily Roundup S...,https://www.fda.gov/news-events/press-announce...,"Mon, 14 Sep 2020 21:00:00 GMT",2020-09-14 21:00:00+00:00
